问题：一些无聊的幼稚黑客在你的网站页面表单中输入文本”pýtĥöñ”，然后你想将这些字符清理掉

In [15]:
# 比如，你可能想消除整个区间上的字符或者去除变音符。 为了这样做，你可以使用经常会被忽视的 str.translate() 方法。
s = 'pýtĥöñ\fis\tawesome\r\n'
s

'pýtĥöñ\x0cis\tawesome\r\n'

In [29]:
# 第一步清理空白字符串。先创建一个小的转换表格然后使用translate()方法：
# 转换表可用通过str.maketrans建立
# 注意这里可以通过字典形式建立转换表，但是字典的键必须是unicode 序列数，不能为字符串本身。
remap = {
    ord('\t'): ' ',
    ord('\f'): ' ',
    ord('\r'): None  # 删除
}
a = s.translate(remap)
a

'pýtĥöñ is awesome\n'

In [33]:
# 删除所有和音符：
import unicodedata
import sys

# 方法1：先标准化，再转换
t1 = unicodedata.normalize('NFD', a)
result = ''.join(c for c in t1 if not unicodedata.combining(c))
result


'python is awesome\n'

In [34]:
# 方法2：
# 这里的思路是先对文本做一些初步的清理， 然后再结合 encode() 或者 decode() 操作来清除或修改它。
b = unicodedata.normalize('NFD', a)
b.encode('ascii', 'ignore').decode('ascii')

# 补充说明encode()第二参数：
# 第二个参数控制错误处理的策略，默认的参数就是strict，代表遇到非法字符时抛出异常； 
# 如果设置为ignore，则会忽略非法字符； 
# 如果设置为replace，则会用?取代非法字符； 
# 如果设置为xmlcharrefreplace，则使用XML的字符引用。 

# 这里的标准化操作将原来的文本分解为单独的和音符。接下来的ASCII编码/解码只是简单的一下子丢弃掉那些字符。 
# 当然，这种方法仅仅只在最后的目标就是获取到文本对应ACSII表示的时候生效。


'python is awesome\n'

In [38]:
# 方法3：
# sys.maxunicode,如果输出的值为65535,那么就是UCS-2,如果输出是1114111就是UCS-4编码
# 通过使用 dict.fromkeys() 方法构造一个字典，每个Unicode和音符作为键，对应的值全部为 None 
cmb_chars = dict.fromkeys(c for c in range(sys.maxunicode) if unicodedata.combining(chr(c)))
b = unicodedata.normalize('NFD', a)
b
b.translate(cmb_chars)

'python is awesome\n'

In [40]:
# 从性能上看，直接使用replace清理字符串效率会比translate快。但是对于复杂情况来说，translate会好点。